In [1]:
from IPython.display import Image

- strategy of generation：生成的策略，李宏毅
    - https://drive.google.com/file/d/1Ac3oFUtq6ThokrMvB7VUfBCUFsoMPba-/view
- paper
    - https://github.com/lucidrains/speculative-decoding/tree/main
    - https://pytorch.org/blog/hitchhikers-guide-speculative-decoding/

In [2]:
Image(url='./speculative_decoding.png', width=800)

- 投机采样（猜测、随机)
- approximation model vs. target model
    - 预言家，模型小，速度快；
        - 一次可以快速生成多个tokens
    - non-AutoRegressive models，一次同时生成多个 tokens
    - compressed models，量化/蒸馏，压缩过的小模型；
    - 以外挂的形式接入，而不需要改变LLM；
- 假如输入是 inputs，其输出为连续的两个 token：A, B
    - 则对于 target model (LLM)，可以并行做三次推理，一次生成3个token（A+B+C）
        - target_model(inputs) => A
        - target_mdoel(inputs + A) => B
        - target_mdoel(inputs + A + B) => C
- 预言家犯错的情况，inputs => A,B (A is right, B is wrong)
    - target_model(inputs) => A
    - target_model(inputs + A) => **C**（这里识别错误）
    - target_model(inputs + A + B) => D (显然更不可信)
    - 这样依然可以一次生成2个tokens（A 和 C）
- 极限情况下预言家预测的tokens （A 和 B）全错
    -  target_model(inputs) => C
    -  target_model(inputs + A) => D
    -  target_mdoel(inputs + A + B) => E
    -  此时最差可以拿到一个争取的 token；